## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:

# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,46.40,76,40,26.46,scattered clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,77.00,94,77,4.00,broken clouds
2,2,Jamestown,US,42.0970,-79.2353,23.00,68,90,17.27,overcast clouds
3,3,La Ronge,CA,55.1001,-105.2842,-20.20,63,39,9.22,scattered clouds
4,4,Bambanglipuro,ID,-7.9500,110.2833,80.55,76,100,4.47,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puerto Ayora,EC,-0.7393,-90.3518,77.00,94,77,4.00,broken clouds
4,4,Bambanglipuro,ID,-7.9500,110.2833,80.55,76,100,4.47,light rain
5,5,Yaan,NG,7.3833,8.5667,76.77,19,58,4.32,broken clouds
8,8,Alyangula,AU,-13.8483,136.4192,78.80,89,90,16.11,moderate rain
11,11,Bonthe,SL,7.5264,-12.5050,77.07,84,0,7.43,clear sky
15,15,Samarai,PG,-10.6167,150.6667,85.32,70,33,8.34,scattered clouds
19,19,Miranda,CO,3.2499,-76.2281,77.00,69,75,12.66,broken clouds
22,22,Vaini,TO,-21.2000,-175.2000,84.20,89,75,4.61,broken clouds
24,24,Puerto Colombia,CO,10.9878,-74.9547,80.60,78,20,8.05,few clouds
27,27,Avarua,CK,-21.2078,-159.7750,84.20,74,40,13.80,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,77.00,broken clouds,-0.7393,-90.3518,
4,Bambanglipuro,ID,80.55,light rain,-7.9500,110.2833,
5,Yaan,NG,76.77,broken clouds,7.3833,8.5667,
8,Alyangula,AU,78.80,moderate rain,-13.8483,136.4192,
11,Bonthe,SL,77.07,clear sky,7.5264,-12.5050,
15,Samarai,PG,85.32,scattered clouds,-10.6167,150.6667,
19,Miranda,CO,77.00,broken clouds,3.2499,-76.2281,
22,Vaini,TO,84.20,broken clouds,-21.2000,-175.2000,
24,Puerto Colombia,CO,80.60,few clouds,10.9878,-74.9547,
27,Avarua,CK,84.20,scattered clouds,-21.2078,-159.7750,


In [8]:

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

    # 6e. Make request and retrieve the JSON data from the search. 
    params['location'] = f'{str(lat)},{str(lng)}'
    hotels = requests.get(base_url, params=params).json()
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except:
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,77.00,broken clouds,-0.7393,-90.3518,Castillo Galapagos
4,Bambanglipuro,ID,80.55,light rain,-7.9500,110.2833,Puri Brata
5,Yaan,NG,76.77,broken clouds,7.3833,8.5667,
8,Alyangula,AU,78.80,moderate rain,-13.8483,136.4192,Groote Eylandt Lodge
11,Bonthe,SL,77.07,clear sky,7.5264,-12.5050,Bonthe Holiday Village
15,Samarai,PG,85.32,scattered clouds,-10.6167,150.6667,Nuli Sapi
19,Miranda,CO,77.00,broken clouds,3.2499,-76.2281,unidiversidad organica la utopia
22,Vaini,TO,84.20,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
24,Puerto Colombia,CO,80.60,few clouds,10.9878,-74.9547,Hotel El Emigrante
27,Avarua,CK,84.20,scattered clouds,-21.2078,-159.7750,Paradise Inn


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,77.00,broken clouds,-0.7393,-90.3518,Castillo Galapagos
4,Bambanglipuro,ID,80.55,light rain,-7.9500,110.2833,Puri Brata
5,Yaan,NG,76.77,broken clouds,7.3833,8.5667,
8,Alyangula,AU,78.80,moderate rain,-13.8483,136.4192,Groote Eylandt Lodge
11,Bonthe,SL,77.07,clear sky,7.5264,-12.5050,Bonthe Holiday Village
...,...,...,...,...,...,...,...
665,San Nicolas,PH,80.60,few clouds,18.1725,120.5953,NORTHVIEW Hotel
667,Ikot Ekpene,NG,80.02,clear sky,5.1794,7.7108,Four Points Hotel
668,Cockburn Town,TC,76.98,few clouds,21.4612,-71.1419,Osprey Beach Hotel
669,Kapasan,ID,79.99,light rain,-6.7773,111.3789,Perum Perhutani Kph Mantingan


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))